In [1]:
from DiSuQ.Torch import models
from torch import tensor
from numpy import arange,linspace,meshgrid,array,log
from DiSuQ.utils import plotCompare,plotHeatmap
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracy,Adam
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(32)

In [2]:
pairs = {'Lx':'Ly','Cx':'Cy','Jx':'Jy','CJx':'CJy'}

In [3]:
basis = [10,10,10] ; rep = 'Q'
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,sparse=False,symmetry=True)

In [4]:
components = circuit.circuitComposition()
L0 = indE(1e-7); print('Inductance Bound(GHz):',L0)
components['Lx'].L0 = L0 ; components['Ly'].L0 = L0
C0 = capE(1e-15); print('Capacitance Bound(GHz):',C0)

Inductance Bound(GHz): 1.6346151294983178
Capacitance Bound(GHz): 19.370229307707593


In [5]:
lossFunction = lossDegeneracy
iterations = 5

In [6]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)}]

In [7]:
Algo = ["lrBFGS",'Nelder-Mead','LBFGS']

In [8]:
def multiInitSearch(optimizer,initials,subspace):
    lrBFGS,NelMea,LBFGS = [],[],[]
    for index,parameter in enumerate(initials):
        print(optimizer.circuitState())
        print(index,parameter)
        optimizer.circuit.initialization(parameter)  
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
        LBFGS.append(optimizer.minimization(lossFunction,flux_profile,
                                                   method='L-BFGS-B',tol=1e-6,maxiter=iterations))

        optimizer.circuit.initialization(parameter)
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
        NelMea.append(optimizer.minimization(lossFunction,flux_profile,subspace=subspace,
                                                 method='Nelder-Mead',tol=1e-6,maxiter=iterations))

        optimizer.circuit.initialization(parameter)
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
        lrBFGS.append(optimizer.optimizationLBFGS(lossFunction,flux_profile,iterations=iterations,lr=.1))        

        return lrBFGS,NelMea,LBFGS       

In [9]:
def optimizationAnalysis(init,subspace,Search,success=1.):
    Loss,Success,Paths = [],[],[]
    for index,(init,(dLogs,dParams,dCircuit)) in enumerate(zip(init,Search)):
        Paths.append(dCircuit[subspace].to_numpy())
        loss = dLogs['loss'].to_numpy()
        Loss.append(loss[-1])
        if loss[-1] < success:
            Success.append(len(loss))
    return Paths,Loss,Success

In [10]:
def analysisPlotting(Optimization):
    paths = dict()
    losse = dict()
    for algo,(Paths,Loss,Success) in zip(Algo,Optimization):
        indices = argsort(Loss)[0]
        paths[algo] = Paths[indices]
        #for idx in indices:
        #    paths[algo+'-'+str(idx)] = Paths[idx]
        losse[algo] = Loss
    return paths,losse

In [11]:
def lossScapeBounds(paths):
    Paths = []
    for algo,path in paths.items():
        Paths.append(path)
    Paths = vstack(Paths)
    Paths = Paths[:, ~isnan(Paths).any(axis=0)]
    return Paths.min(0),Paths.max(0)

In [12]:
N = 3; subspace = ('Lx','Cx','Jx','CJx')
initials = uniformParameters(circuit,subspace,N)
#initials = truncNormalParameters(circuit,subspace,N,var=5)
len(initials)

81

In [13]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

In [ ]:
Optimization = multiInitSearch(optimizer,[initials[18]],subspace)
lrBFGS,NelMea,LBFGS = Optimization

{'Lx': 1.3621787729789503e-05, 'Ly': 1.3621787729789503e-05, 'Cx': 0.9999998807907104, 'Cy': 0.9999998807907104, 'Jx': 10.000001907348633, 'Jy': 10.000001907348633, 'CJx': 100.0, 'CJy': 100.0}
0 {'Lx': 1.2259613471237383, 'Ly': 1.3621787729789503e-05, 'Cx': 625.0, 'Cy': 0.9999998807907104, 'Jx': 300.0, 'Jy': 10.000001907348633, 'CJx': 625.0, 'CJy': 100.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.71872D-03    |proj g|=  2.36848D-02


In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in Optimization]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution',export='pdf',size=(600,800))